## Time-based manual segmentation


In [26]:
import os
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import dark

midi_file = os.path.join("data", "inputs", "MIDI5-TRIMMED.mid")

MIDIPlayer(midi_file, 400, styler=dark)

In [27]:
import numpy as np
from pathlib import Path

def divide_midi(input_file, output_folder, segment_length=8):
    filename = Path(input_file).stem
    midi_data = pretty_midi.PrettyMIDI(input_file)
    total_length = midi_data.get_end_time()

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for start in range(0, int(total_length), segment_length):
        end = start + segment_length
        segment = midi_data.get_piano_roll(fs=100, times=np.arange(start, min(end, total_length), 1/100))

        # create a new MIDI object for the segment
        segment_midi = pretty_midi.PrettyMIDI()
        instrument = pretty_midi.Instrument(program=midi_data.instruments[0].program, name=f"{filename}_{start}-{end}")

        # add notes from the original MIDI that fall within the current segment
        for note in midi_data.instruments[0].notes:
            if start <= note.start < end:
                new_note = pretty_midi.Note(
                    velocity=note.velocity,
                    pitch=note.pitch,
                    start=note.start - start,
                    end=min(note.end, end) - start
                )
                instrument.notes.append(new_note)

        # write out
        segment_midi.instruments.append(instrument)
        segment_midi.write(os.path.join(output_folder, f'{filename}_{start}-{end}.mid'))

In [28]:
dir_out = os.path.join("data", "outputs", "time", f"{Path(midi_file).stem}")

divide_midi(midi_file, dir_out)